# 6. CRP + Hawkes process

In [ ]:
# Common imports 
from ast import literal_eval

import gensim
import numpy as np
import pandas as pd
import datetime
import sys
print(sys.version)

## Load and preprocess the data

First, we need to load the data from the csv. This file contains the documents already processed and cleaned after applying the following steps:

1. Tokenization
2. Homogeneization, which includes:
    1. Removing capitalization.
    2. Removing non alphanumeric tokens (e.g. punktuation signs)
    3. Stemming/Lemmatization.
3. Cleaning
4. Vectorization


We load it as a `pandas` dataframe. 

***Note***: The timestamp column contains the time at which the event was generated in ***seconds***.


In [ ]:
df = pd.read_csv('../data/tweets_cleaned.csv')
df.drop_duplicates(subset="tweet", inplace=True)

df['tokens'] = df['tokens'].apply(literal_eval) #Transform the string into a list of tokens
df.sort_values(by='timestamp', inplace=True)

### Create the dictionary

Up to this point, we have transformed the raw text collection in a list of documents stored in `X_tokens`, where each document is a collection 
of the words that are most relevant for semantic analysis. Now, we need to convert these data (a list of token lists) into 
a numerical representation (a list of vectors, or a matrix). To do so, we will start using the tools provided by the `gensim` library. 

As a first step, we create a dictionary containing all tokens in our text corpus, and assigning an integer identifier to each one of them.

In [ ]:
I = 350
X_tokens = list(df['tokens'].values)
dictionary = gensim.corpora.Dictionary(X_tokens)

dictionary.filter_extremes(no_below=15, no_above=0.5, keep_n=I)

I = len(dictionary)

### Create the dataset for the HDHP

Build the dataset `X_events` as a list of events where each event is a tuple `X_events[i] = (ts, w, user_id, meta_info)` of four values:
- `ts`: timestamp of the event. It is important to choose the base time scale (e.g., days, months) and it is also recommended to normalized them so that the timestamp of the first event is 0.
- `w`: represents the content of the document. It should be a string containing the tokens separated by a white space.
- `user_id`: the id of the user that generated the event. They should go from 0 to the number of users contained in the dataset
- `meta_info`: list with extra information. It is not needed for the tutorial so it can be set to `[]`

***Exercise 2.1:***  Implement the code to build the X_event list

In [ ]:
X_events = list()
# TODO

## Hierarchical Dirichlet Hawkes Proces (HDHP)

Import the Hierarchical Dirichlet Hawkes Proces module. Complete details about the code can be found in the official Github repository at https://github.com/Networks-Learning/hdhp.py. There, you can also find an example of how to use the package in the Jupyter notebooks under the directory examples.

***Exercise 1.1***: Find the piece of code  where the $\mu_u$ parameters are updated.  If you cannot find it, explain the reason.

TODO

***Exercise 1.2***: Find the piece of code where the $\alpha_l$ parameters are updated.  If you cannot find it, explain the reason.

TODO

***Exercise 1.3***:  Find the piece of code where the $\theta_l$ parameters are updated. If you cannot find it, explain the reason.

TODO

***Exercise 1.4***: Find the piece of code where the $z_{1:n}$ table's assignment variables are updated.  If you cannot find it, explain the reason.

TODO

***Exercise 1.5***: Explore the code and explain how the final particle is chosen at the last iteration. Recall, we run the SMC with $|P|$ particles but at the end we only consider one sample per parameter to show the results.

TODO

In [ ]:
import hdhp

In [ ]:
vocabulary = list(dictionary.values())
doc_min_length = 2
doc_length = 10
words_per_pattern = 15

alpha_0 = (2.5, 0.75)  # Parameters of the prior for the alpha, i.e,, Gamma distribution
mu_0 = (2, 0.5)  # Parameters of the prior for the mu_u, i.e, Gamma distribution
omega = 1  # Decay of the exponential  kernel, i.e., memory of the intensity.
beta = 0.01 # Concentration parameter for the Dirichlet Process

In [ ]:
particle, norms = hdhp.infer(X_events, alpha_0=alpha_0, mu_0=mu_0, omega=omega, num_particles=15, beta=beta, seed=512)

***Exercise 2.2***: Compute the log-likelihood.

***Exercise 2.3***: From the HDHP code provided extract the necesary information about the learning patterns to fill the requested table.


***Exercise 2.4***:  From the HDHP code provided extract the necesary information about the users to fill the requested table.